# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from ipywidgets.embed import embed_minimal_html

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)
print("done")

done


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#query_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    #'address={0}&key={1}').format(target_city, gkey)
file = "../output_data/cities.csv"
cities_df = pd.read_csv(file)
cities_df.head()

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere
0,geraldton,AU,-28.77,114.60,82.40,14,0,24.16,South
1,iqaluit,CA,63.75,-68.51,35.60,80,90,16.11,North
2,busselton,AU,-33.65,115.33,69.01,38,5,14.76,South
3,bethel,US,41.37,-73.41,60.01,87,1,3.36,North
4,narrabri,AU,-30.32,149.78,66.20,36,0,11.41,South


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
#url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
url = "https://maps.googleapis.com/maps/api/geocode/json?address="
query_url = url + "portland" + "&key=" + g_key
testerz = requests.get(query_url).json()
#print(json.dumps(testerz, indent=4))
#Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
# Fill NaN values and convert to float
# rating = airport_df["Airport Rating"].astype(float)
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
humidity = cities_df['Humidity%'].astype(int)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
# Add layer
fig.add_layer(heatmap_layer)
# Display figure
embed_minimal_html('../maps/citiesbyhumid.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
cities_df.head()

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere
0,geraldton,AU,-28.77,114.60,82.40,14,0,24.16,South
1,iqaluit,CA,63.75,-68.51,35.60,80,90,16.11,North
2,busselton,AU,-33.65,115.33,69.01,38,5,14.76,South
3,bethel,US,41.37,-73.41,60.01,87,1,3.36,North
4,narrabri,AU,-30.32,149.78,66.20,36,0,11.41,South


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
hotel_df = cities_df.loc[
    (cities_df["Max Temp(F)"]>70) &
    (cities_df["Max Temp(F)"]<80) &
    (cities_df["Windspeed(MPH)"]<10) &
    (cities_df["Cloudiness%"]==0)]
hotel_df = hotel_df.reindex(columns=hotel_df.columns.tolist() + ["Hotel Name"])
hotel_df

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
16,jalu,LY,29.03,21.55,73.98,44,0,5.39,North,NaN
57,airai,TL,-8.93,125.41,78.33,37,0,4.88,South,NaN
86,riyadh,SA,24.69,46.72,75.20,23,0,4.70,North,NaN
209,vao,NC,-22.67,167.48,71.60,69,0,8.84,South,NaN
254,aldama,MX,22.92,-98.07,79.00,91,0,2.75,North,NaN
326,do gonbadan,IR,30.36,50.80,75.20,29,0,4.59,North,NaN
394,abu kamal,SY,34.45,40.92,72.36,48,0,7.16,North,NaN
399,isla vista,US,34.41,-119.86,75.00,89,0,4.61,North,NaN
438,kabinda,CD,-6.13,24.48,70.03,79,0,2.77,South,NaN
474,alice springs,AU,-23.70,133.88,75.00,18,0,8.05,South,NaN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
locales = (hotel_df[['Latitude','Longitude']])

In [9]:
for index, row in locales.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location":f"{row[0]},{row[1]}",
        "radius":5000,
        "type":"lodging",
        "key":g_key
    }
    response = requests.get(url,params=params).json()
    results = response["results"]
    
    try:
        hotel_df.loc[index,"Hotel Name"] = results[0]["name"]
    except (KeyError,IndexError):
        pass
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Latitude,Longitude,Max Temp(F),Humidity%,Cloudiness%,Windspeed(MPH),Hemisphere,Hotel Name
16,jalu,LY,29.03,21.55,73.98,44,0,5.39,North,Jalu Hotel
57,airai,TL,-8.93,125.41,78.33,37,0,4.88,South,Juvinal Martin
86,riyadh,SA,24.69,46.72,75.20,23,0,4.70,North,InterContinental Riyadh
209,vao,NC,-22.67,167.48,71.60,69,0,8.84,South,Hotel Kou-Bugny
254,aldama,MX,22.92,-98.07,79.00,91,0,2.75,North,CASA CANYE
326,do gonbadan,IR,30.36,50.80,75.20,29,0,4.59,North,مهمانسرا و تالار فرهنگیان شهرستان گچساران
394,abu kamal,SY,34.45,40.92,72.36,48,0,7.16,North,محطة التلفاز
399,isla vista,US,34.41,-119.86,75.00,89,0,4.61,North,Super 8 by Wyndham Santa Barbara/Goleta
438,kabinda,CD,-6.13,24.48,70.03,79,0,2.77,South,Village Mianda Mutole
474,alice springs,AU,-23.70,133.88,75.00,18,0,8.05,South,Desert Palms Alice Springs


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [19]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
embed_minimal_html('../maps/hotels.html', views=[fig])
fig

Figure(layout=FigureLayout(height='420px'))